In [12]:
import pandas as pd
import json
from collections import defaultdict

In [3]:
with open('../static/data/co99_d00.json') as json_data:
    data = json.load(json_data)

In [ ]:
data

### Scope the polygons to just the NCR counties

In [214]:
scoped = defaultdict(list)
scoped['type'] =  'FeatureCollection'

for polygon in data['geometries']:
    #list of all coords in polygon
    points = polygon['coordinates'][0]
    
    #list of longs/lats for all coords
    longs = [point[0] for point in points]
    lats = [point[1] for point in points]
    
    #scope using max and mins of lats/longs and add to dictionary
    if max(longs) < -76.6 and max(lats) < 39.36 and min(lats) > 38.3 and min(longs) > -78:
        scoped['features'].append({'type': 'Feature', 'geometry': polygon})

print(len(scoped['features']))
        


15


In [143]:
#write dictionary to a json file
file_location = '/Users/mikenlee/Documents/Personal/analytics/GWU/county_expenditures/static/data/scoped_counties.geojson'

json.dump(scoped, open(file_location, 'w'))

In [ ]:
scoped

## Add `properties` information to the NCR counties geojson

In [243]:
updated_counties = []
test_counties = []
print(len(scoped['features']))
for county in scoped['features']:

    #list of all coords in polygon
    coords = county['geometry']['coordinates'][0]
    
    #list of longs/lats for all coords
    longs = [coord[0] for coord in coords]
    lats = [coord[1] for coord in coords]

#     print(f'{min(longs)} #####> -77.16')
#     print(f'{max(longs)} #####< -77.11')
#     print(f'{max(lats)} #####< -39.14')
#     print(f'{min(lats)} #####> 39.13')
    
    #select using max and mins of lats/longs and add to dictionary
    #MONTGOMERY
    
#     if -77.197759 in longs and 39.340466 in lats:
#         county.update({'properties': {'county': 'Montgomery County', 'population': 123456}})
#         updated_counties.append(county)
        
    if max(lats) < 39.5 and min(lats) > 38.91:
        county.update({'properties': {'county': 'Montgomery County', 'population': 123456}})
        updated_counties.append(county)
        

    #LOUDOUN
    elif max(lats) < 39.3289 and min(lats) > 38.84372 and min(longs) > -77.967 and max(longs) < -77.319:
        county.update({'properties': {'county': 'Loudoun County', 'population': 123456}})
        updated_counties.append(county)

    #ARLINGTON
    elif max(lats) < 38.935 and min(lats) > 38.825 and min(longs) > -77.173 and max(longs) < -77.027:
        county.update({'properties': {'county': 'Arlington County', 'population': 123456}})
        updated_counties.append(county)
        
    #PRINCE GEORGE'S
    elif max(lats) < 39.139 and min(lats) > 38.532 and min(longs) > -77.1 and max(longs) < -76.63:
        county.update({'properties': {'county': "Prince George's County", 'population': 123456}})
        updated_counties.append(county)
        
    #PRINCE WILLIAM
    elif -77.675857 in longs and 38.900693 in lats:
        county.update({'properties': {'county': 'Prince William County', 'population': 123456}})
        updated_counties.append(county)
        
    #FAIRFAX COUNTY
    elif -77.394985 in longs and 38.990391 in lats:
        county.update({'properties': {'county': 'Fairfax County', 'population': 123456}})
        updated_counties.append(county)

    #DC
    elif -77.013798 in longs and 38.97441 in lats:
        county.update({'properties': {'county': 'DC', 'population': 123456}})
        updated_counties.append(county)

    #Alexandria City
    elif -77.07787 in longs and 38.800127 in lats:
        county.update({'properties': {'county': 'Alexandria City', 'population': 123456}})
        updated_counties.append(county)

    #Fairfax
    elif -77.303406 in longs and 38.834912 in lats:
        county.update({'properties': {'county': 'Fairfax', 'population': 123456}})
        updated_counties.append(county)
        
    #Manassas
    elif -77.517607 in longs and 38.742749 in lats:
        county.update({'properties': {'county': 'Manassas', 'population': 123456}})
        updated_counties.append(county)
    
    #Falls Church
    elif -77.149301 in longs and 38.873212 in lats:
        county.update({'properties': {'county': 'Falls Church', 'population': 123456}})
        updated_counties.append(county)

    #Manassas Park
    elif -77.44211 in longs and 38.775814 in lats:
        county.update({'properties': {'county': 'Manassas Park', 'population': 123456}})
        updated_counties.append(county)

    #Manassas Park
    elif -77.439219 in longs and 38.766926 in lats:
        county.update({'properties': {'county': 'Manassas Park', 'population': 123456}})
        updated_counties.append(county)

    #Fairfax
    elif -77.314422 in longs and 38.846377 in lats:
        county.update({'properties': {'county': 'Fairfax', 'population': 123456}})
        updated_counties.append(county)
    
    #Manassas
    else:
        county.update({'properties': {'county': "Manassas", 'population': 123456}})
        updated_counties.append(county)
        
    
print(len(updated_counties))        
# print(updated_counties)
print(len(test_counties))   
test_counties

15
15
0


[]

In [247]:
new_scoped = defaultdict(list)
new_scoped['type'] =  'FeatureCollection'

new_scoped['features'] = updated_counties


In [310]:
for county in new_scoped['features']:
    print(county['properties']['county'])

new_scoped['features'][0]['properties']

Montgomery County
Loudoun County
Prince George's County
Fairfax County
DC
Prince William County
Arlington County
Falls Church
Fairfax
Fairfax
Alexandria City
Manassas Park
Manassas
Manassas Park
Manassas


{'county': 'Montgomery County', 'population': 123456}

### Scrape Wikipedia for county information

In [315]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import json
import unicodedata

In [251]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/Users/mikenlee/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


In [312]:
alexandria_url = 'https://en.wikipedia.org/wiki/Alexandria,_Virginia'
loudoun_url = 'https://en.wikipedia.org/wiki/Loudoun_County,_Virginia'
montgomery_url = 'https://en.wikipedia.org/wiki/Montgomery_County,_Maryland'
arlington_url = 'https://en.wikipedia.org/wiki/Arlington_County,_Virginia'
princewilliam_url = 'https://en.wikipedia.org/wiki/Prince_William_County,_Virginia'
princegeorges_url = "https://en.wikipedia.org/wiki/Prince_George's_County,_Maryland"
dc_url = 'https://en.wikipedia.org/wiki/Washington,_D.C.'
fallschurch_url = 'https://en.wikipedia.org/wiki/Falls_Church,_Virginia'
manassas_url = 'https://en.wikipedia.org/wiki/Manassas,_Virginia'
manassaspark_url = 'https://en.wikipedia.org/wiki/Manassas_Park,_Virginia'
fairfax_url = 'https://en.wikipedia.org/wiki/Fairfax_County,_Virginia'
fairfaxcity_url = 'https://en.wikipedia.org/wiki/Fairfax,_Virginia'

url_list = [alexandria_url, loudoun_url, montgomery_url, arlington_url, princewilliam_url, princegeorges_url, dc_url,
            fallschurch_url, manassas_url, manassaspark_url, fairfax_url, fairfaxcity_url]

In [253]:
browser.visit(fairfax_url)

In [254]:
soup = BeautifulSoup(browser.html, parser='html.parser')

In [306]:
soup.find('div', class_="fn org").text

'Fairfax County'

In [317]:
for url in url_list: 
    browser.visit(url)
    soup = BeautifulSoup(browser.html, parser='html.parser')
    # get all html rows
    trows = soup.find_all('tr')
    #get wikicounty name
    wiki_countyname = soup.find('div', class_="fn org").text

    #if the visited wiki page matches the county, update dictionary
    for i, county in enumerate(new_scoped['features']):
        
        #check if county matches the wiki page
        if county['properties']['county'] in wiki_countyname:

            #find specifc th's to extract td text and update county properties GEOJSON
            for row in trows:
                try:
                    if row.th.text == 'Founded':
                        new_scoped['features'][i]['properties']['founded'] = unicodedata.normalize('NFKD', row.td.text)
                    if row.th.text == 'Named for':
                        new_scoped['features'][i]['properties']['named_for'] = unicodedata.normalize('NFKD', row.td.text)
                    if row.th.text == 'Seat':
                        new_scoped['features'][i]['properties']['seat'] = unicodedata.normalize('NFKD', row.td.text)
                    if 'Largest' in row.th.text:
                        new_scoped['features'][i]['properties']['largest_town'] = unicodedata.normalize('NFKD', row.td.text)
                    if 'Total' in row.th.text and 'mi' in row.td.text:
                        new_scoped['features'][i]['properties']['total_area'] = unicodedata.normalize('NFKD', row.td.text)
                    if 'Land' in row.th.text and 'mi' in row.td.text:
                        new_scoped['features'][i]['properties']['land_area'] = unicodedata.normalize('NFKD', row.td.text)
                    if 'Water' in row.th.text:
                        new_scoped['features'][i]['properties']['water_area'] = unicodedata.normalize('NFKD', row.td.text)
                    if 'Estimate' in row.th.text:
                        new_scoped['features'][i]['properties']['population'] = unicodedata.normalize('NFKD', row.td.text)
                    if 'Density' in row.th.text:
                        new_scoped['features'][i]['properties']['density'] = unicodedata.normalize('NFKD', row.td.text)

                except:
                    continue
        
        #if visited wiki page doesn't match the county go to the next county
        else:
            continue

for county in new_scoped['features']:
    print(county['properties'])
    print('--------')

{'county': 'Montgomery County', 'population': '1,050,688', 'founded': 'September 6, 1776[3][4]', 'named_for': 'Richard Montgomery', 'total_area': '507 sq mi (1,310 km2)', 'land_area': '491 sq mi (1,270 km2)', 'water_area': '16 sq mi (40 km2)  3.1%', 'density': '1,900/sq mi (740/km2)'}
--------
{'county': 'Loudoun County', 'population': '413,538', 'founded': '1757', 'named_for': 'John Campbell, 4th Earl of Loudoun[2]', 'seat': 'Leesburg', 'largest_town': 'Leesburg', 'total_area': '521 sq mi (1,350 km2)', 'land_area': '516 sq mi (1,340 km2)', 'water_area': '6 sq mi (20 km2)  1.1%', 'density': '600/sq mi (230/km2)'}
--------
{'county': "Prince George's County", 'population': 123456}
--------
{'county': 'Fairfax County', 'population': '1,147,532', 'founded': 'June 19, 1742', 'named_for': 'Thomas Fairfax, 6th Lord Fairfax of Cameron', 'seat': 'Fairfax (independent city)1', 'largest_town': 'Herndon', 'total_area': '406 sq mi (1,050 km2)', 'land_area': '\nFairfax County Government Center\nWol

In [258]:
pd.read_html(f'<table>{info_table.tbody}</table>')

[                                       Fairfax County  \
 0                                              County   
 1                                   County of Fairfax   
 2                                Mount Vernon mansion   
 3                                        FlagSealLogo   
 4          Location within the U.S. state of Virginia   
 5                 Virginia's location within the U.S.   
 6   Coordinates: .mw-parser-output .geo-default,.m...   
 7                                             Country   
 8                                               State   
 9                                             Founded   
 10                                          Named for   
 11                                               Seat   
 12                                       Largest town   
 13                                               Area   
 14                                            • Total   
 15                                             • Land   
 16           

In [248]:
#write dictionary to a json file
file_location = '/Users/mikenlee/Documents/Personal/analytics/GWU/county_expenditures/static/data/scoped_counties_info.geojson'

json.dump(new_scoped, open(file_location, 'w'))